In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import plotly.graph_objects as go
from scipy.stats import norm

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

from haystack_utils import get_mlp_activations
from hook_utils import get_ablate_neuron_hook
import haystack_utils
from pythia_160m_utils import get_neuron_accuracy, ablation_effect
from plotting_utils import plot_neuron_acts, color_binned_histogram


%reload_ext autoreload
%autoreload 2

model = HookedTransformer.from_pretrained("tiny-stories-33M",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device="cuda")

Using pad_token, but it is not set yet.


Loaded pretrained model tiny-stories-33M into HookedTransformer


In [4]:
dataset = load_dataset('roneneldan/TinyStories', split='train')
print(dataset[0])

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'text': 'One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on her shirt.\n\nLily went to her mom and said, "Mom, I found this needle. Can you share it with me and sew my shirt?" Her mom smiled and said, "Yes, Lily, we can share the needle and fix your shirt."\n\nTogether, they shared the needle and sewed the button on Lily\'s shirt. It was not difficult for them because they were sharing and helping each other. After they finished, Lily thanked her mom for sharing the needle and fixing her shirt. They both felt happy because they had shared and worked together.'}


In [19]:
print(model.generate("John is high and Sarah is low. John is in a tree and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is in a treehouse and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on top of a pole and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on top of a house and Sarah", 100))
print(model.generate("John is high and Sarah is low. John is on the roof and Sarah", 100))


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is in a tree and Sarah is under his tree. He is stuck. John is scared. He says: "Help! Help!" Sarah hears him and moves off to another tree. John says: "Thank you, Sarah!" Sarah is happy to help.

But then Sarah hears a worried noise. She looks up and sees a bird cage. The cage is very high and Sarah's heart is beating very fast. She wants to help the bird. She quickly takes off the cushion and the cushage. She puts the


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is in a treehouse and Sarah is on the swing.

One day, they hear an alarm. John is scared and Sarah is curious. She asks John to do a test. John asked questions like, 'What is powerful?' and 'What should the alarm do?' Sarah thought for a minute and then she said 'I think it is because of the alarm'.

John felt much better and said to Sarah, “I think I understand now! Let’s find out who is causing the alarm and make


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on top of a pole and Sarah is on the ground. They both frowning and Sarah's frownss.

John was angry. He wanted to get down and shout. He thought of another way. He didn't want to stay up there with Sarah alone.

John had an idea. He took his scooter and pushed Sarah off the floor. Sarah fell on the floor with a loud clatter. She was mad.

John and Sarah started to quarrel. They yelled at each other and Sarah had to


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on top of a house and Sarah is close behind. Sarah is backing around and making noises.

"I want to have fun now," she says to John.

John is not happy. He said: "No, it's dangerous to go underneath the tree."
Sarah starts to cry.

But John mustered up the courage and said: "Come on, Sarah. Let's go under the tree."

So they both crawled under the tree and found a beautiful meadow full of flowers. They


  0%|          | 0/100 [00:00<?, ?it/s]

John is high and Sarah is low. John is on the roof and Sarah is on the grass. John smiles and thinks this is a good idea. He makes a hole in the balloon and then he takes out a plastic stick. He cuts the string to make it. Sarah watches and asks why John does this when he stops climbing.

John looks back onto the ground. Sarah says, "Next time, I will help you climb the pile of leaves". John smiles. He likes to help Sarah. They work together and it is fun.
<|endoftext|>


In [21]:
print(model.generate("John is low and Sarah is high. John is on the ground and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is in a hole and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is underneath a pile of leave and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is under the house and Sarah", 100))
print(model.generate("John is low and Sarah is high. John is under the horse and Sarah", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is on the ground and Sarah is on the ground. They are very sad. 

They see a big chain in the park. They both like chains. They want to pick it up and play with it.

They both pull on it and it breaks. They are very sad and the chain is gone. 

John and Sarah take the chain and go home. They hope to find the chain again someday. That was very special.
<|endoftext|>


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is in a hole and Sarah is not there.

"Where is Sarah?" John asked.

"I don't know," Sarah replied.

John kept searching and searching. He looked all around the hole, but he could not find Sarah.

"I am not in the hole!" he yelled out.

Suddenly a big purple spider appeared. It began to chase John. He ran and ran until he reached home.

The spider caught John and he was taken to the hospital. Sarah


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is underneath a pile of leave and Sarah is playing with her dolls. She sees John and smiles. She wants to be friends with John.

John's mom hears Sarah's voice and calls out "Emily, where are you?" She looks under the pile but she can not see Sarah.

Jane was happy to see Tim but she is still scared of his toes. Then she saw something on Sarah's toes. It was a bug! She stared at it. But then it scurries away.

Sarah said,


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is under the house and Sarah is still under it. Sarah cries and blinks her eyes. She is sad and misses John. She hopes that her pain will go away.

John hears Sarah crying. He feels bad for her and goes to her. He is slow and careful. He makes room for her out of the under. He takes her deeper from the hole and eats her candy. He makes her smile. He hopes that she will feel better soon.

Sarah's arm gets better after a few minutes.


  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. John is under the horse and Sarah is under the horse. John is thirsty and the horse drinks from Sarah. Sarah is sad and she starts to cry.

Then suddenly, the horse stops eating. John is shocked and he wants to get Sarah out. But he does not know how to do it.

John is upset and he cries. Sarah feels bad about being left out. She is sorry that she should have been alone. Sarah needs to go back to the stable and John has to chase her.

Sarah


In [24]:
print(model.generate("John is low and Sarah is high. Sarah is up a tree and John is", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

John is low and Sarah is high. Sarah is up a tree and John is at the bottom. He could see many things from up there. Sarah was on the swings and John could see them from up high. He was complaining because he was too small to reach them.

John kept complaining and Sarah looked away. Then she saw a man. He was carrying a big bag and looked easyening his way into the tree. When he got there he talked to mommy. He said he could get her over there because Sarah was too small.

Mommy thanked


In [29]:
print(model.generate("", 100))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack and Sarah are in the meadow. One of them is in a tree. The other is a girl named Sophie.

John looks up and sees Sophie. He points at her and says, "She is teasing me!" Sarah ran to John and says, "Sorry, John. That's not nice at all."

John and Sarah climb the tree and sit on a branch. They look at the sky and the fluffy clouds that moved across the sky. Sarah pointed at the clouds and said, "Look John, I see a cloud that looks like a dog!"

John


In [56]:
# Selecting multi-use objects in the environment
print(model.generate("Jack was thirsty. Jack saw a pond, so he", 100, temperature=0))
print(model.generate("Jack was bored. Jack saw a pond, so he", 100, temperature=0))

print(model.generate("Jack was sleepy. Jack saw a mat, so he", 100, temperature=0))
print(model.generate("Jack was excited. Jack saw a mat, so he", 100, temperature=0))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack was thirsty. Jack saw a pond, so he went to take a drink. He saw a big frog and wanted to catch it. He tried to catch the frog, but it jumped away. Jack was sad, but he kept walking.

Suddenly, Jack saw a big dog. The dog barked at Jack and he got scared. He tried to run away, but the dog chased him. Jack ran and ran, but the dog was too fast. The dog caught Jack and bit him. Jack cried and cried.

The end


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was bored. Jack saw a pond, so he decided to go for a swim. He jumped in and started to splash around. He was having so much fun!

Suddenly, Jack heard a noise. He looked around and saw a big frog. The frog was jumping around and making a lot of noise. Jack was scared and he wanted to get out of the water.

Jack started to swim towards the frog. The frog saw Jack and started to swim away. Jack was sad and he started to cry.

Jack's mom


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was sleepy. Jack saw a mat, so he lay down on it. He closed his eyes and started to sleep.

Suddenly, Jack heard a noise. He opened his eyes and saw a big, scary monster! Jack was so scared he couldn't move.

The monster said, "I'm going to eat you!"

Jack was so scared he couldn't move. He just lay there, too scared to move.

The monster grabbed Jack and ate him up. Jack was never seen again.
<|endoftext|>


  0%|          | 0/100 [00:00<?, ?it/s]

Jack was excited. Jack saw a mat, so he ran over to it. He picked it up and started to play. He rolled it around and laughed. He was having so much fun.

Suddenly, Jack heard a noise. He looked around and saw a big dog. The dog was barking and running towards him. Jack was scared. He tried to run away, but the dog was too fast.

The dog bit Jack's leg. Jack screamed and cried. He was hurt. The dog ran away.

Jack was sad


In [63]:
# multi-use environment elements
# print(model.generate("The ball rolled off the bridge", 100, temperature=0))
# print(model.generate("Put the ball onto the hill. The ball", 100, temperature=0))
# print(model.generate("Sarah put the box onto the hill. The box", 100, temperature=0))
# print(model.generate("The box was too big and heavy. Sarah pushed the box ", 100, temperature=0))




# print(model.generate("Jack was tired. Jack saw a slide, so he", 100, temperature=0))
# print(model.generate("Jack was thinking. Jack saw a slide, so he", 100, temperature=0))
# print(model.generate("Jack was near the pond, but he was thirsty so he ", 100, temperature=0))
# print(model.generate("Jack was near the pond, but he wasn't thirsty so he ", 100, temperature=0))
# print(model.generate("Jack was far away so he ", 100, temperature=0))
# print(model.generate("Jack was close by so he ", 100, temperature=0))
# print(model.generate("Jack was above Sarah so he ", 100, temperature=0))
# print(model.generate("Jack was a long way from home and he felt thirsty so he", 100, temperature=0))



  0%|          | 0/100 [00:00<?, ?it/s]

The box was too big and heavy. Sarah pushed the box and it started to roll. She was so happy! She ran after the box and it rolled down the street.

Sarah followed the box until it stopped at a park. She saw a big tree and she wanted to climb it. She put her foot on the trunk and started to climb. She was so excited!

But then she heard a loud noise. It was the park keeper. He was angry and he shouted at Sarah. He said she was not allowed to climb the tree


In [4]:
print(model.generate("Jack wanted to draw on the ground. Jack saw a stick. He ", 100, temperature=0))

print(model.generate("Jack wanted to draw on the ground. Jack saw a stick. He use ", 100, temperature=0))


  0%|          | 0/100 [00:00<?, ?it/s]

Jack wanted to draw on the ground. Jack saw a stick. He was very excited. He picked it up and started to draw on the ground. He drew a big circle and then a small square. He was very happy.

Jack's mom saw him and said, "Jack, what are you drawing?"

Jack said, "I'm drawing a circle. Look, it's so big and round!"

Jack's mom smiled and said, "That's great! You're so creative!"

Jack smiled and kept drawing. He was


In [60]:
print(model.generate("Once upon a time there lived a girl named Lily. She loved to look in her attic. One day, she found a pond in her attic. She ", 100, temperature=0))


  0%|          | 0/100 [00:00<?, ?it/s]

Once upon a time there lived a girl named Lily. She loved to look in her attic. One day, she found a pond in her attic. She was so excited to see it! She ran to the pond and saw that it was full of water. She wanted to play in it, but her mom said no.

Lily was sad, but then she had an idea. She asked her mom if she could soak her feet in the pond. Her mom said yes, so Lily put her feet in the water and it felt so nice. She splashed around and had so much fun.

After a while, Lily's mom


In [73]:
print(model.generate("Jack was playing in the park. But", 20, temperature=0))
print(model.generate("Jack was in the pool. But", 20, temperature=0))
print(model.generate("Jack was going to school. But", 20, temperature=0))
print(model.generate("Jack was sad. But", 20, temperature=0))
print(model.generate("Jack was sleeping in bed. But", 20, temperature=0))
print(model.generate("Jack was lying in bed. But", 20, temperature=0))

  0%|          | 0/20 [00:00<?, ?it/s]

Jack was playing in the park. But then he saw something strange. He saw a big, scary dog. It was barking and growling


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was in the pool. But he was not happy. He was very angry. He wanted to play with his friends, but they


  0%|          | 0/20 [00:00<?, ?it/s]

...

Jack was sad. But then he saw a big, shiny motorcycle. It was so cool! He wanted to ride it.


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was sleeping in bed. But he was not sleeping. He was dreaming about a big, scary monster. The monster was coming to


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was lying in bed. But he was not feeling very good. He was feeling very tired. He had been playing all day and


In [79]:
print(model.generate("Jack was a boy who loved to draw. He drew red drawings with his crayon and blue drawings with his pencil. He drew a blue circle with his", 20, temperature=0))
print(model.generate("Jack was a boy who loved to draw. He drew blue drawings with his crayon and red drawings with his pencil. He drew a blue circle with his", 20, temperature=0))


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was a boy who loved to draw. He drew red drawings with his crayon and blue drawings with his pencil. He drew a blue circle with his pencil and then drew a green triangle with his red pencil. He was very proud of his drawing.


  0%|          | 0/20 [00:00<?, ?it/s]

Jack was a boy who loved to draw. He drew blue drawings with his crayon and red drawings with his pencil. He drew a blue circle with his pencil and then drew a green triangle with his blue pencil. He was very happy with his drawing.


In [22]:
print(model.generate(" The park was closed. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" The park was large. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" The park was very good. Jack asked his mother whether he could go to the park.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a cat.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a dog.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a rock.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold an ant.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold an elephant.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a star.", 20, temperature=0))
print(model.generate(" Jack asked his mother whether he could hold a rocket.", 20, temperature=0))


# Basic properties of objects

  0%|          | 0/20 [00:00<?, ?it/s]

 The park was closed. Jack asked his mother whether he could go to the park. His mother said, "No, it's too late. You need to stay home." Jack was


  0%|          | 0/20 [00:00<?, ?it/s]

 The park was large. Jack asked his mother whether he could go to the park. His mother said, "Yes, but you must stay close to me."

Jack was so


  0%|          | 0/20 [00:00<?, ?it/s]

...

 Jack asked his mother whether he could hold an elephant. His mother said, "No, Jack. Elephants are too big and need to be in


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack asked his mother whether he could hold a star. His mother said, "No, Jack. Stars are too far away."

Jack was sad


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack asked his mother whether he could hold a rocket. His mother said, "No, Jack. Rockets are dangerous." Jack was sad, but he listened


In [64]:
# print(model.generate(" Jack had three apples. Jack ate one of his apples. Now Jack had", 20, temperature=0))

# print(model.generate(" Sarah had three pears. Sarah ate two pears. Then she had", 20, temperature=0))

# print(model.generate(" Jack picked up the apple but not left the star on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the rocket but he left the orange on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the orange but he left the rocket on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the star but he left the orange on the ground. Jack held the", 20, temperature=0))
# print(model.generate(" Jack picked up the orange but he left the apple on the ground. Jack held the", 20, temperature=0))

# print(model.generate(" Jack held the apple but not the star. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the rocket but not the orange. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the orange but not the rocket. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the star but not the orange. Jack held the", 20, temperature=0))
# print(model.generate(" Jack held the orange but not the apple. Jack held the", 20, temperature=0))


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the apple but not the star. Jack held the apple tight and said, "I don't want to share this apple with you."

His


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the rocket but not the orange. Jack held the rocket and it felt so light. He was so happy. He wanted to show his friends.



  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the orange but not the rocket. Jack held the rocket and it felt so light. He was so happy. He wanted to show his friends.



  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the star but not the orange. Jack held the star and looked at it. He was so happy. He wanted to show it to his mom.


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack held the orange but not the apple. Jack held the apple and looked at it. He wanted to eat it, but he knew he had to be careful


In [5]:
print(model.generate("Jack asked his mother whether he could have an icecream. His mother said", 100, temperature=0))

  0%|          | 0/100 [00:00<?, ?it/s]

Jack asked his mother whether he could have an icecream. His mother said, "No, Jack. You need to eat something healthy first."

Jack was sad and he started to cry. His mother said, "Don't worry, Jack. I will make you an ice cream. It will be yummy and healthy."

Jack was happy and he stopped crying. He said, "Yes, please!"

His mother made him an ice cream and Jack ate it. He was so happy and he said, "Thank you, Mommy!"



In [44]:
# prompt = "Jack said, \"Can I have a frog?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack was going away and, \"Can I have a frog?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack am, \"Am I alive?.\""
# print(model.generate(prompt, 100, temperature=0))


# prompt = "Jack said, \"Why is red?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack said, \"Why is we alive?.\""
# print(model.generate(prompt, 100, temperature=0))

# prompt = "Jack said, \"Why is the cow fast?.\""
# print(model.generate(prompt, 100, temperature=0))

prompt = "Jack said, \"why tree blue?.\""
print(model.generate(prompt, 100, temperature=0))




  0%|          | 0/100 [00:00<?, ?it/s]

Jack said, "why tree blue?."

His mom said, "The tree is green. It is a special tree."

Jack said, "I want to climb it!"

His mom said, "No, it is too high. You can't climb it."

Jack said, "But I want to climb it!"

His mom said, "No, it is too dangerous. You can't climb it."

Jack said, "But I want to climb it!"

His mom


In [65]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his mother Sally.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his mother Sally. He was so excited and he wanted to get it.

Jack asked his mother if he could have some ice cream. His mother said yes and she gave him a big bowl of ice cream. Jack was so happy and he ate it all up.

But then Jack started to feel sick. He had eaten too much ice cream and his tummy hurt. He started to cry and his mother was very worried. She took him to the hospital and the doctor said he had to stay in the hospital for a few days.

Jack was very sad and he was very sorry for not listening to his mother. He learned that it is important to listen to your parents and not eat too much ice cream.
<|endoftext|>


In [66]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Jack was so excited and he wanted to get the ice cream.

Jack asked Sarah if he could have some ice cream. Sarah said no and told Jack to go away. Jack was very sad and he started to cry.

Suddenly, a big man came and he was very angry. He said to Jack, "You must leave the park now!" Jack was scared and he ran away.

Jack was so sad that he didn't get the ice cream. He was very sad and he never got to eat the ice cream.
<|endoftext|>


In [69]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sally, but he had to go home.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sally, but he had to go home.

Jack was sad because he wanted to eat the ice cream. He asked Sally if he could have some, but she said no. Jack was very sad and he started to cry.

Suddenly, a big, scary man came and he was very angry. He said that Jack was not allowed to have any ice cream. He said that he would punish Jack if he didn't leave the park.

Jack was scared and he ran away. He was so sad that he didn't get any ice cream. He was very sad and he never went to the park again.
<|endoftext|>


In [67]:
print(model.generate(" Once upon a time there was a girl named Sarah and a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah.", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a girl named Sarah and a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. He asked Sarah if he could have some. Sarah said yes and gave Jack a big scoop of ice cream.

Jack was so happy and he wanted to show Sarah how much he liked the ice cream. He asked Sarah if he could have some more. Sarah said yes and gave Jack a big scoop of ice cream. Jack was so happy and he thanked Sarah for the ice cream.

Jack and Sarah had a lot of fun at the park. They played on the swings and the slide. They even had a picnic with some of the ice cream. Jack was so happy and he thanked Sarah for the ice cream.

Jack and Sarah had a great time at the park. They both agreed that it was the best day ever!
<|endoftext|>


In [68]:
print(model.generate(" Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Sarah gave him some icecream. Jack was very happy. ", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time there was a boy named Jack. Jack was going to the park and he wanted icecream. He saw some icecream next to his friend Sarah. Sarah gave him some icecream. Jack was very happy. 

Jack and Sarah went to the park and Jack saw a big tree. He wanted to climb it. He started to climb the tree but it was too hard. He was too scared. 

Then Sarah said, "Let's go to the ice cream shop. I will give you some ice cream." Jack was so happy. He said, "Thank you Sarah!" 

They went to the ice cream shop and Jack got his ice cream. He was so happy. He ate it all up. 

Jack and Sarah had a great day at the park. They were both very happy.
<|endoftext|>


In [72]:
print(model.generate(" Once upon a time, Jack and Sarah lived by a chair. One day, they both wanted to sit in the chair at the same time. They didn't want to sit together. Sarah sat in the chair. Jack ", 200, temperature=0))


  0%|          | 0/200 [00:00<?, ?it/s]

 Once upon a time, Jack and Sarah lived by a chair. One day, they both wanted to sit in the chair at the same time. They didn't want to sit together. Sarah sat in the chair. Jack was getting angry. He said, "I want to sit in the chair!" Sarah said, "No, I want to sit in the chair!" They started to argue and it made them both very sad.

Suddenly, Jack had an idea. He said, "Let's both sit in the chair together!" Sarah thought that was a great idea. They both sat in the chair together and they were happy again. They realized that sharing was better than fighting. From that day on, they always shared the chair when they were together.
<|endoftext|>


In [22]:
print(model.to_tokens(' eating'))
print(model.to_tokens(' eat'))

tensor([[50256,  6600]], device='cuda:0')
tensor([[50256,  4483]], device='cuda:0')


In [19]:
continuous_prompts = [
    'Sarah loved to',
    'Sarah did not want to',
    'Sarah hated to',
    'Jack liked to',
    'Jack enjoyed going',
    'Jack and Sarah were',
    'While mom was cooking, Jack was',
    'While Sarah was eating, Jack was']

simple_prompts = [
    'Sarah went to'
    'Sarah went to play'
    'Sarah went to a play'
    'When Jack picked up a cat, Sarah picked up a dog',
]

for prompt in continuous_prompts:
    print(model.generate(prompt, 20))


for prompt in simple_prompts:
    print(model.generate(prompt, 200))


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah loved to play with her toys, especially her toy automobile. She would take it everywhere she went. One day


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah did not want to talk to the pink dress. She said the pink dress was too pretty. The lady was sad and


  0%|          | 0/20 [00:00<?, ?it/s]

...

While Sarah was eating, Jack was looking at his dark plate. "Mommy, why did my plate look dark?" Jack asked.


  0%|          | 0/200 [00:00<?, ?it/s]

Sarah went toShe was very excitied to her friend John. They decided to go camping in the woods. Sarah felt excited about the camp they were about. 

The night before going to sleep, they took a small camping blanket and layed down on the camp. John was keeping a close pace, his steps leading the way. Sarah was very good at navigating the fast way.

John ran past her and arrived at a small cottage. "Interesting!" he said to Sarah. She was sitting at her end of the day. The night was warm and the sun was shining. Joe said goodnight and waved goodbye.

Suddenly, Sarah's pet dog, Ginger, trotted off by! Ginger followed Sarah, but Sarah had a worried look – what was outside the house at the end of the night? Sarah quickly opened the door. She welcomed Ginger safely.
<|endoftext|>


  0%|          | 0/200 [00:00<?, ?it/s]

Sarah went to playSarah went to a playWhen Jack picked up a cat, Sarah picked up a dog called Max. They were both very excited and ready to go play.

The game was to play catch with the intelligent dog, the cat was the one who tried to catch. All the older kids and the cat felt discouraged and were trying very hard.

But the dog was very smart and by the end of the day, he had caught all the cat and made it back back to the place they were playing. Everyone cheered with joy and soon Jack and Sarah were playing with the intelligent dog. They both felt happy and enjoyed playing with the dog.
<|endoftext|>


In [3]:
print(model.generate(" Jack said, \"Do cows eat grass?\"", 200, temperature=0))


  0%|          | 0/200 [00:00<?, ?it/s]

 Jack said, "Do cows eat grass?" His mom said, "No, cows don't eat grass. They eat grass." Jack said, "I want to eat grass." His mom said, "No, Jack. We can't eat grass. We have to go to the store." Jack said, "Okay, Mom." They went to the store and Jack saw a big pile of green grass. He said, "Mom, can I eat the grass?" His mom said, "No, Jack. We have to buy the grass." Jack said, "Okay, Mom." They bought the green grass and went home. Jack ate the grass and said, "Yum, grass is so good!"
<|endoftext|>


In [23]:
print(model.generate(" Jack met Sarah and asked, \"Is that cow in front of me?\" His mom said,", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Jack walked past a cow and asked, "Is that cow in front of me?" His mom said, "No, cows can't be. But they are very big and strong." Jack said, "I want to be big and strong like the cow." His mom said, "You are already big and strong, my little boy."

Later that day, Jack saw a man with a big truck. The man said, "I need to load some heavy boxes onto my truck." Jack asked, "What are you doing?" The man said, "I am loading boxes onto my truck." Jack said, "I want to help you." The man said, "Thank you, little boy."

The man loaded the boxes onto the truck and drove away. Jack was happy to help the man and learn about the big and strong cow.
<|endoftext|>


In [25]:
print(model.generate(" Jack saw a brown cow and asked, \"Is that cow green?\" His mom said,", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Jack saw a brown cow and asked, "Is that cow green?" His mom said, "No, that cow is brown like the grass." Jack was happy to hear that and continued to play.

Later, Jack's mom asked him, "Did you have fun playing outside?" Jack said, "Yes, I saw a brown cow and a brown cow!" Jack's mom smiled and said, "That sounds like fun. Let's go inside and have some milk and cookies." Jack was excited to go inside and eat some milk and cookies.
<|endoftext|>


In [27]:
print(model.generate(" Jack saw Sarah and asked, \"Is that my friend?\" His mom said,", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Jack saw Sarah and asked, "Is that my friend?" His mom said, "No, it's not your friend. But don't worry, we can go to the park and play." Jack was happy and said, "Yay! I love the park!"

At the park, Jack saw a big slide. He wanted to go down it, but he was scared. His mom said, "It's okay, Jack. You can do it. Just hold on tight." Jack took a deep breath and went down the slide. He felt the wind in his hair and it was so much fun!

After playing, Jack and his mom went to get some ice cream. Jack said, "Mom, can I have a chocolate ice cream?" His mom said, "Sure, Jack. You can have a chocolate ice cream." Jack was so happy and said, "Yay! I love ice cream!"
<|endoftext|>


In [30]:
print(model.generate(" Jack", 20, temperature=0))
print(model.generate(" Jill", 20, temperature=0))
print(model.generate(" Once", 20, temperature=0))
print(model.generate(" Once", 20, temperature=0))

  0%|          | 0/20 [00:00<?, ?it/s]

 Jack and Jill were best friends. They liked to play together every day. One day, Jack had a


  0%|          | 0/20 [00:00<?, ?it/s]

 Jill and Tom were best friends. They liked to play together every day. One day, Jill had a


  0%|          | 0/20 [00:00<?, ?it/s]

 Once upon a time, there was a little girl named Lily. She loved to play outside in the sunshine


In [41]:
# print(model.generate(" The", 100, temperature=0))
# print(model.generate(" A", 100, temperature=0))
# print(model.generate(" An", 100, temperature=0))
print(model.generate(" Jack was high in a tree. Sarah was beneath", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Jack was high in a tree. Sarah was beneath the tree and she was very happy. She looked around and saw a big, red apple. She wanted to eat it, but it was too high for her to reach.

Suddenly, a big bird flew down and said, "I can help you get the apple." Jack was scared, but he said, "Okay, let's try."

The bird flew up to the apple and picked it off the tree. He handed it to Sarah and said, "Here you go."

Sarah was so happy. She said, "Thank you, bird!"

The bird smiled and said, "You're welcome." Then he flew away.

Sarah took the apple and ate it. It was so yummy! She was so happy that she had the apple.
<|endoftext|>


In [42]:
print(model.generate(" Jack asked, \"What color are frogs?\"", 200, temperature=0))

  0%|          | 0/200 [00:00<?, ?it/s]

 Jack asked, "What color are frogs?" His mom said, "Frogs are green and brown." Jack said, "I like frogs!" His mom said, "Let's go to the pond and see if we can find some frogs."

When they got to the pond, Jack saw a frog. He said, "Mom, look! A frog!" His mom said, "Yes, it's a green frog." Jack said, "Can I catch it?" His mom said, "No, it's too fast. Let's just watch it."

Jack and his mom watched the frog for a while. Then, Jack said, "I want to catch it!" His mom said, "No, it's too fast. Let's just watch it." Jack said, "Okay."

They watched the frog for a few more minutes. Then, Jack said, "I want to catch it!" His mom said, "No, let's not catch it. Let's just watch it." Jack said


In [47]:
# print(model.to_tokens(" Once upon a time"))
# print(model.generate(" Jack ", 300))




tensor([[50256,  4874,  2402,   257,   640]], device='cuda:0')


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack asked his mom, "Can I have a sad?" His mom replied, "I don't have the answer for you, but I can make you something


In [35]:
# print(model.generate(" Jack asked his mom, \"Can I have a sad?\"", 20))
# print(model.generate(" Jack asked his mom, \"Can I have a bad?\"", 20))
# print(model.generate(" Jack asked his mom, \"Can I have a happy?\"", 20))

def rand_hook(value, hook):
    value = value * 0.0
    return value

continuous_prompts = [
    'Sarah loved to',
    'Sarah did not want to',
    'Sarah hated to',
    'Jack liked to',
    'Jack enjoyed going',
    'Jack and Sarah were',
    'While mom was cooking, Jack was',
    'While Sarah was eating, Jack was']

# for prompt in continuous_prompts:
prompt = "Sarah"
print(model.generate(prompt, 20, temperature=0))
for layer in range(model.cfg.n_layers):
    with model.hooks([(f'blocks.{layer}.hook_attn_out', rand_hook)]):
        print(model.generate(prompt, 20, temperature=0))
    with model.hooks([(f'blocks.{layer}.hook_mlp_out', rand_hook)]):
        print(model.generate(prompt, 20, temperature=0))
# break

  0%|          | 0/20 [00:00<?, ?it/s]

Sarah was a little girl who loved to play outside. One day, she went to the park with her


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah. Lily said her big very, Lily had a little frightened.









  0%|          | 0/20 [00:00<?, ?it/s]

...

Sarah was a little girl named Lily. She loved to play outside in the sunshine. One day, she


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah and her mommy went to the park. They saw a big tree with a lot of leaves.


  0%|          | 0/20 [00:00<?, ?it/s]

Sarah was a little girl who loved to play outside. She always wanted to make her own special things with


In [19]:
print(model.generate(" Jack did not want to play. Sarah asked, \"Do you want to play?\"", 20, temperature=0))
# print(model.generate(" Jack did not want to eat. Sarah asked, \"Do you want to eat?\"", 20))

def rand_hook(value, hook):
    value = value * 0.0
    return value

with model.hooks([('blocks.1.mlp.hook_post', rand_hook)]):
    print(model.generate(" Jack did not want to play. Sarah asked, \"Do you want to play?\"", 20, temperature=0))


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack did not want to play. Sarah asked, "Do you want to play?" Jack said, "No, I don't want to play." Sarah said, "Why not?"


  0%|          | 0/20 [00:00<?, ?it/s]

 Jack did not want to play. Sarah asked, "Do you want to play?" Jack said, "No, I want to play alone." 

Jack felt sad because he


In [2]:
model = HookedTransformer.from_pretrained("roneneldan/TinyStories-1M",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device="cuda")

Using pad_token, but it is not set yet.


Loaded pretrained model roneneldan/TinyStories-1M into HookedTransformer


In [3]:
random_prompts = [
    "Tim found a green frog in his garden. He named it Fred. Fred liked to jump and play in the pond.",
    "Sally had a magic paintbrush. Anything she painted became real. She painted a big rainbow, and it",
    "Tom loved to look at the stars. One night, a shooting star landed in his yard. It was a starship!",
    "Amy found a little lost kitten in the park. She took it home and fed it milk. The kitten purred ",
    "In Billy's backyard, there was a magic tree. If you whispered a wish to it, the wish came true! ",
    "One rainy day, Lisa and her brother raced raindrops down the window. They cheered for their ",
    "Jenny planted a sunflower seed. She watered it and watched it grow. It grew taller and taller. ",
    "Sam built a kingdom of sandcastles at the beach. There were big castles and little castles. Sam",
    "Tina let go of her red balloon by accident. It floated up to the sky. Tina watched it drift away",
    "Molly danced under the moon with her teddy bear. They twirled and leaped. The stars watched and ",
    "Ben saw a bear in his backyard! It was big and furry but friendly. The bear liked honey and fish.",
    "Sara heard the wind whisper her name. It told her stories of faraway lands and secret treasures. ",
    "Leo found a lake that laughed when you told it jokes. He told it knock-knock jokes, and the lake ",
    "One day, the sun slept in, and it was dark in the morning. People were confused, but the birds",
    "Kelly had a pair of shoes that could talk! They told her stories of where they'd been and what",
    "In a snowy land, penguins had a parade every year. They dressed up, waddled in lines, and danced ",
    "One fluffy cloud took a day off from the sky. It floated down to visit a little girl named Emily.",
    "Frank had a fish friend in a bowl. They couldn't talk, but they understood each other. Frank ",
    "There was a scarecrow that danced at night. When everyone was asleep, it would twirl, leap, and",
    "Anna found a slide at the end of a rainbow. She climbed up and slid down, right through colors ",
    "Timmy had a hat that could travel through time. He visited dinosaurs, pirates, and even went to ",
    "In the woods, there was a rock that snored. People would come and listen. Some thought it was ",
    "Lily had a mirror that laughed at silly faces. The more faces she made, the harder it laughed.",
    "Bella the butterfly had a birthday. All her insect friends came. They had nectar cake and",
    "Harry had a hat that talked. It told him jokes and sang songs. Harry wore the hat to school,",
    "Every night, the moon asked for a story. Children would tell tales of brave heroes and magical ",
    "Olivia met a friendly dolphin in the ocean. They swam together and jumped waves. Olivia told the",
    "There was a tree that hugged people. If you were sad or happy, it would give you a big, wooden hug.",
    "Eddie had a magic eraser. It could erase anything, even rain clouds or bad days. Eddie used it to",
    "Peter was a polite pirate. He said please and thank you. He found treasure but always shared. Other",
    "A rabbit found glasses in the woods. When he put them on, he could see colors he never saw before. ",
    "Lucy built a castle in the clouds. She went there to read and dream. The clouds were soft, and the ",
    "Tim the turtle raced with the rabbits. They were fast, but Tim was slow and steady. The rabbits ",
    "In a magical meadow, the flowers played music. When the wind blew, they sang sweet songs. Children ",
    "Jenny had jellybeans that jumped! They bounced around and made funny noises. Jenny and her friends ",
    "In a big tree lived a wise old owl. He knew everything and helped everyone. If you had a question or ",
    "Sam put on a sock puppet show. His socks became kings, queens, and clowns. The whole family watched ",
    "At night, a cat danced on the rooftops. It twirled and leaped under the stars. Only a little girl",
    "Billy's bath was full of bubbles. When he jumped in, the bubbles started to talk! They told funny ",
    "A dog dreamed of a land full of bones and toys. He dug and played all day. When he woke up, he found ",
    "Sally had a sunflower that smiled. It smiled at the sun and at Sally. When Sally was sad, the sunflower ",
    "In an old house lived a friendly ghost. It didn't scare people; it made friends. It played with children ",
    "Tommy found a tiny dinosaur in his garden. It was small and liked to eat lettuce. Tommy and the dinosaur ",
    "Benny the Bear had a honey party. All the forest animals came, and they ate honey cakes and honey pies. They danced and laughed all night. Benny's honey party was the sweetest ever.",
    "Sara caught a snowflake, and it became her friend. They danced in the snow and made snow angels. When the sun came out, the snowflake had to go, but it promised to come back next winter.",
    "Marty the Mouse went on a big adventure. He explored the garden and found cheese mountains and cracker bridges. He was a brave mouse explorer, and he came home with tales to tell.",
    "Ella the Elephant had a paintbrush. She painted colorful pictures with her trunk. People came from far and wide to see her art. Ella was happy to share her colorful world.",
    "Tom's toothbrush talked when it was brushing time. It told jokes and sang songs about clean teeth. Tom loved brushing time, and his teeth sparkled and sang too.",
    "In Daisy's garden, the daisies danced at dawn. They twirled to the bird's song and swayed in the breeze. Daisy danced with them, and they were all dancing daisies together.",
    "Freddy the Fish wished to fly. A friendly bird carried him up, and Freddy flew over the sea! He saw whales and waves. Then he splashed back home, a fish that had flown.",
    "Randy the Rabbit found a rainbow in his burrow. He hopped through colors and found a land of carrot gold. Randy nibbled and played, then hopped back home, the richest rabbit ever.",
    "Cindy was best friends with a cloud. They played shadow games and rained on flowers. Cindy loved her cloudy friend, and they floated through many happy days together.",
    "Sammy the Snail raced with the beetles. They buzzed and dashed, but Sammy was slow and sure. The beetles got tired, but Sammy kept going. He won the race and was the snail champ!",
]

longer_stories = [
    "Bobby Bear loved picnics. One sunny day, he packed his basket with honey sandwiches, apple juice, and chocolate cookies. He invited all his teddy friends, and they went to the big green park. They laughed, played hide-and-seek, and ate all the yummy food. Bobby Bear's picnic was the best, and they all clapped their paws and danced in the sun.",

    "Ducky loved rainy days. When the clouds gathered, and the first drops fell, Ducky would quack with joy. He splashed in puddles, jumped with the frogs, and made waves with his yellow feet. The rain was like music, and Ducky danced to the tune. The other ducks joined in, and they all had a splashy, quacky, rainy day of fun.",

    "Kitty had a magical colorful tail. It was red, blue, yellow, and even had sparkles. Everywhere Kitty went, her tail wagged and made pretty shapes in the air. The other cats were amazed, and the children loved to watch Kitty's tail. Kitty became a star, and her colorful tail was famous in the whole town. Kitty was proud and happy to have such a special tail.",

    "Timmy went on a train ride. The train went choo-choo, and Timmy's eyes grew wide. He saw cows, trees, mountains, and rivers. The train went fast, and it went slow. Timmy ate popcorn and waved at the people. The train ride was the most exciting adventure, and Timmy wanted to ride the train every day.",

    "Lulu had a big round balloon. It was pink and floated like a cloud. Lulu danced with the balloon in her room. They twirled, they floated, they jumped. Lulu and the balloon were the best of friends. They even had tea parties and told secrets. Lulu's balloon was her special friend, and they were happy together all the time.",

    "Billy loved to dance with bumblebees. He would go to the garden, and the bumblebees would come. They buzzed and twirled, and Billy danced with them. The flowers clapped, and the sun smiled. Billy was the bee dance king, and his garden was the happiest place. Everyone wanted to see Billy's bumblebee dance, and he was always ready to twirl.",

    "Sally made a snowman. It was big and round with a carrot nose and two big eyes. Sally and the snowman played all day in the snow. They made snow angels, they threw snowballs, and they laughed. When the sun went down, Sally gave the snowman a big hug and said goodnight. They were the best of snowy friends, and Sally couldn't wait for more snow.",

    "Tommy had a turtle named Tilly. Tilly was slow and green, but she was the best friend ever. Tommy and Tilly had races in the yard. Tommy would run, and Tilly would crawl. They both reached the finish line, and they both won! Tommy cheered, and Tilly smiled. They were the best racing team, and they loved their turtle time together.",

    "Jenny had jelly that jumped. It was red jelly, and it loved to boing and boing. Jenny would laugh and catch the jelly. It jumped into her bowl, and they had a jelly feast. Jenny's friends came to see the jumping jelly, and they all played together. Jenny's jumping jelly was the yummiest and funniest jelly ever, and they all wanted more.",

    "Danny's dog had dreams. At night, it would bark and wag its tail in its sleep. Danny wondered what it was dreaming. Maybe a big bone, maybe a park full of friends, maybe a never-ending bowl of doggy treats. Danny smiled at his dreaming dog and wished it sweet dreams. They were the best of friends, and Danny's dog's dreams were always happy.",

    "Milly looked at the moon every night. It was big, bright, and beautiful. Milly said goodnight to the moon and the stars. She told them stories and sang them songs. The moon and the stars twinkled back at her. They were Milly's nighttime friends, and they watched over her as she slept. Milly's moon gazing was her special time, and she loved the moon and the stars.",

    "Freddy Frog had a pond party. All the pond friends came. The ducks quacked, the fish splashed, the turtles sunbathed. Freddy Frog was the host, and he jumped and croaked. They played games and danced in the water. Freddy Frog's pond was the best party place, and they all had a splashing good time. The pond was alive with fun, and Freddy Frog was the happiest frog ever.",

    "Lily had a lemon that laughed. It was a funny lemon with a big smile. Lily squeezed the lemon, and it giggled. Hee-hee! They made lemonade together, and the lemonade was giggly too. Lily and her laughing lemon were the best of friends, and they made every day a sunny, lemony, happy day."
]


In [113]:
for layer in range(model.cfg.n_layers):
    acts = haystack_utils.get_mlp_activations(random_prompts, layer, model, mean=True, disable_tqdm=True)
    max_values, max_indices = torch.max(acts, dim=0)
    print(max_values, max_indices)

tensor(0.1119, device='cuda:0') tensor(109, device='cuda:0')
tensor(0.2005, device='cuda:0') tensor(213, device='cuda:0')
tensor(0.3224, device='cuda:0') tensor(233, device='cuda:0')
tensor(0.1366, device='cuda:0') tensor(149, device='cuda:0')
tensor(0.1526, device='cuda:0') tensor(4, device='cuda:0')
tensor(0.0796, device='cuda:0') tensor(197, device='cuda:0')
tensor(0.0898, device='cuda:0') tensor(48, device='cuda:0')
tensor(0.1738, device='cuda:0') tensor(101, device='cuda:0')


In [117]:
for layer in range(model.cfg.n_layers):
    acts = haystack_utils.get_mlp_activations(random_prompts, layer, model, mean=True, disable_tqdm=True)
    max_values, max_indices = torch.max(acts, dim=0)
    print(max_values, max_indices)

tensor(0.1082, device='cuda:0') tensor(109, device='cuda:0')
tensor(0.2025, device='cuda:0') tensor(213, device='cuda:0')
tensor(0.3126, device='cuda:0') tensor(233, device='cuda:0')
tensor(0.1346, device='cuda:0') tensor(149, device='cuda:0')
tensor(0.1519, device='cuda:0') tensor(4, device='cuda:0')
tensor(0.0743, device='cuda:0') tensor(197, device='cuda:0')
tensor(0.0862, device='cuda:0') tensor(48, device='cuda:0')
tensor(0.1665, device='cuda:0') tensor(101, device='cuda:0')


In [109]:
from hook_utils import save_activation
with model.hooks([('blocks.2.mlp.hook_post', save_activation)]):
    model(random_prompts[0])
acts = model.hook_dict['blocks.2.mlp.hook_post'].ctx['activation']

haystack_utils.clean_print_strings_as_html(model.to_str_tokens(random_prompts[0]), acts[0, :, 233], max_value=0.7)

In [17]:
model.cfg.n_layers

8

In [15]:
import plotting_utils
large_acts_df = plotting_utils.get_neuron_moments(model, random_prompts + longer_stories,
                                                  [[i, j] for i in range(8) for j in range(256)], hook_pre=False)

# plotting_utils.plot_neuron_acts(model, random_prompts + longer_stories, [[0, 109], [1, 213], [2, 233], [3, 149], [4, 4], [5, 197], [6, 48], [7, 88],])

In [16]:
print(large_acts_df.nlargest(5, 'skew'))
print(large_acts_df.nlargest(5, 'kurtosis'))

layer_neurons = large_acts_df.sort_values('kurtosis', ascending=False).iloc[:5][['layer', 'neuron']].values.tolist()

plotting_utils.plot_neuron_acts(model, random_prompts + longer_stories, layer_neurons, range_x=[-0.5, 1])
plotting_utils.plot_neuron_acts(model, random_prompts + longer_stories, layer_neurons, range_x=[-0.5, 1], hook_pre=True)

      layer  neuron      skew   kurtosis
1884      7      92  3.388569  13.586270
1856      7      64  2.998611  11.051443
2021      7     229  2.752743   9.018811
1245      4     221  2.678183   7.012342
1607      6      71  2.632190   9.407434
      layer  neuron      skew   kurtosis
1884      7      92  3.388569  13.586270
1727      6     191  2.569419  12.045842
1856      7      64  2.998611  11.051443
413       1     157  2.581644  11.026487
1369      5      89  2.510441  10.848992


In [105]:
n = 3

print(random_prompts[n])
model.generate(random_prompts[n], 220)

Amy found a little lost kitten in the park. She took it home and fed it milk. The kitten purred 


  0%|          | 0/220 [00:00<?, ?it/s]

'Amy found a little lost kitten in the park. She took it home and fed it milk. The kitten purred \n" swoop down and pounced!" Amy said. Dave opened the purple fur and touched a little bit better. \nAnna helped the kitten talk to someone\'s pet. Then one little one of her collar ripped. The kitten looked miserable but smiled and ran away. \nAmy hugged the baby bird, smiled from ear. She kept her tray close and plant the flower and they all shared it together. \nThe moral of the story is that it\'s better to share something special with everyone in the same place.\n<|endoftext|>'